<a href="https://colab.research.google.com/github/sfarrukhm/modernbert-finetuning/blob/main/token_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset

raw_datasets = load_dataset("conll2003")

README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [3]:
from transformers import AutoTokenizer

model_checkpoint = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer.is_fast

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

True

In [4]:
def align_labels_with_tokens_v1(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels


Some researchers prefer to attribute only one label per word, and assign -100 to the other subtokens in a given word. This is to avoid long words that split into lots of subtokens contributing heavily to the loss. Change the previous function to align labels with input IDs by following this rule.

In [5]:
def align_labels_with_tokens_v2(labels, word_ids):
    new_labels=[]
    prev_id=0
    for new_id in word_ids:
      if new_id ==None:
        label=-100
        new_labels.append(label)


      elif new_id!=prev_id:
        label=labels[new_id]
        new_labels.append(label)
      else:
        label=-100
        new_labels.append(label)
      prev_id=new_id

    return new_labels


Now process the whole dataset. i.e tokenize the dataset and assign token to each new token.
<br>Note: Words in the 'tokens' column in the dataset are not the actual tokens, these are just sentences split into words.

In [6]:
def tokenize_and_align_labels(examples):
  tokenized_inputs=tokenizer(
      examples['tokens'], truncation=True, is_split_into_words=True)
  all_labels=examples['ner_tags']
  new_labels=[]
  for i, labels in enumerate(all_labels):
    word_ids=tokenized_inputs.word_ids(i)
    new_labels.append(align_labels_with_tokens_v1(labels, word_ids))
  tokenized_inputs['labels']=new_labels
  return tokenized_inputs

To take advantage of the speed of our fast tokenizer, it’s best to tokenize lots of texts at the same time, so we’ll write a function that processes a list of examples and use the Dataset.map() method with the option batched=True.

In [7]:
tokenized_datasets=raw_datasets.map(tokenize_and_align_labels,
                                    batched=True,
                                    num_proc=2,
                                    remove_columns=raw_datasets['train'].column_names)

Map (num_proc=2):   0%|          | 0/14041 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/3250 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/3453 [00:00<?, ? examples/s]

Note that we haven’t padded our inputs yet; we’ll do that later, when creating the batches with a data collator.

Now define the training loop

### Data Collation

In [8]:
from transformers import DataCollatorForTokenClassification
data_collator=DataCollatorForTokenClassification(
    tokenizer=tokenizer
)

### Metric
To have the `Trainer` compute a metric every epoch, we will need to define
compute_metric() function
- compute_metric() function will take arrays of predictions ground labels
and return a dictionary of metric name and values
```

In [9]:
# tradional framework to evaluate token classification prediction
# is seqeval library
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=ba5c6a478219380fbed4257d86137c53b89a4e458d875722eb2ede9f8b4c2514
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [10]:
ner_feature = raw_datasets["train"].features["ner_tags"]
label_names = ner_feature.feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [11]:
import evaluate

metric = evaluate.load("seqeval")
labels = raw_datasets["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

In [12]:
predictions = labels.copy()
predictions[2] = "O"
# metric.compute(predictions=[predictions], references=[labels])
print(labels)
print(predictions)

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']
['B-ORG', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O', 'O']


In [13]:
metric.compute(predictions=[predictions], references=[labels])


{'MISC': {'precision': 1.0,
  'recall': 0.5,
  'f1': 0.6666666666666666,
  'number': 2},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 0.6666666666666666,
 'overall_f1': 0.8,
 'overall_accuracy': 0.8888888888888888}

In [19]:
import torch


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    logits = torch.from_numpy(logits)
    predictions = torch.argmax(logits, dim=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

### Define the Model

In [15]:
!pip install triton

In [16]:
id2label={i:label for i,label in enumerate(label_names)}
label2id={label:i for i,label in enumerate(label_names)}

from transformers import AutoModelForTokenClassification
model=AutoModelForTokenClassification.from_pretrained(
    pretrained_model_name_or_path=model_checkpoint,
    id2label=id2label,
    label2id=label2id
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
model.config

BertConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "google-bert/bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-PER",
    "2": "I-PER",
    "3": "B-ORG",
    "4": "I-ORG",
    "5": "B-LOC",
    "6": "I-LOC",
    "7": "B-MISC",
    "8": "I-MISC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-LOC": 5,
    "B-MISC": 7,
    "B-ORG": 3,
    "B-PER": 1,
    "I-LOC": 6,
    "I-MISC": 8,
    "I-ORG": 4,
    "I-PER": 2,
    "O": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.47.1",
  "type_vocab_size": 2,
  "u

Training Loop

In [20]:
from transformers import TrainingArguments
output_dir="/content/drive/MyDrive/huggingface/model_trained/modernbert-conll-ner"
args = TrainingArguments(
    output_dir,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True
)

from transformers import Trainer
trainer=Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    compute_metrics=compute_metrics,
    processing_class=tokenizer,
    data_collator=data_collator
)
trainer.train()
trainer.push_to_hub(commit_message="Training complete")

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.023000,0.068283,0.920079,0.941602,0.930716,0.985941
2,0.022200,0.061393,0.934535,0.951363,0.942874,0.987418
3,0.009700,0.064867,0.935885,0.950690,0.943229,0.987926


events.out.tfevents.1737176230.0f9658e8028d.457.1:   0%|          | 0.00/9.44k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/sfarrukh/modernbert-conll-ner/commit/67c0d2ecc8aaa9538b8ec71fe2dd512654f91ec9', commit_message='Training complete', commit_description='', oid='67c0d2ecc8aaa9538b8ec71fe2dd512654f91ec9', pr_url=None, repo_url=RepoUrl('https://huggingface.co/sfarrukh/modernbert-conll-ner', endpoint='https://huggingface.co', repo_type='model', repo_id='sfarrukh/modernbert-conll-ner'), pr_revision=None, pr_num=None)